## Q1

In [ ]:
import chess
import chess.engine
import heapq

class BeamNode:
    def __init__(self, board, move_sequence, score):
        self.board = board
        self.move_sequence = move_sequence
        self.score = score

    def __lt__(self, other):
        return self.score > other.score

def evaluate_board(board):
    # A simple material evaluator
    values = {
        chess.PAWN: 1,
        chess.KNIGHT: 3,
        chess.BISHOP: 3,
        chess.ROOK: 5,
        chess.QUEEN: 9,
        chess.KING: 0
    }
    score = 0
    for piece_type in values:
        score += len(board.pieces(piece_type, chess.WHITE)) * values[piece_type]
        score -= len(board.pieces(piece_type, chess.BLACK)) * values[piece_type]
    return score if board.turn == chess.WHITE else -score

def beam_search(board, beam_width=3, depth_limit=3):
    initial_node = BeamNode(board.copy(), [], evaluate_board(board))
    beam = [initial_node]

    for depth in range(depth_limit):
        next_beam = []
        for node in beam:
            legal_moves = list(node.board.legal_moves)
            for move in legal_moves:
                new_board = node.board.copy()
                new_board.push(move)
                score = evaluate_board(new_board)
                new_node = BeamNode(new_board, node.move_sequence + [move], score)
                next_beam.append(new_node)
        beam = heapq.nlargest(beam_width, next_beam)

    best_node = max(beam, key=lambda x: x.score)
    return best_node.move_sequence, best_node.score

board = chess.Board()
beam_width = 3
depth_limit = 2

best_moves, score = beam_search(board, beam_width, depth_limit)

print("Best Move Sequence:")
for move in best_moves:
    print(move)

print("Evaluation Score:", score)


Best Move Sequence:
b1c3
b8c6
Evaluation Score: 0


## Q2

In [4]:
import random
import math

def distance(p1, p2):
    return math.hypot(p1[0] - p2[0], p1[1] - p2[1])

def total_distance(route):
    return sum(distance(route[i], route[(i + 1) % len(route)]) for i in range(len(route)))

def generate_neighbors(route):
    neighbors = []
    for i in range(len(route)):
        for j in range(i + 1, len(route)):
            neighbor = route[:]
            neighbor[i], neighbor[j] = neighbor[j], neighbor[i]
            neighbors.append(neighbor)
    return neighbors

def hill_climb(locations, max_iterations=1000):
    current_route = locations[:]
    random.shuffle(current_route)
    current_distance = total_distance(current_route)

    for _ in range(max_iterations):
        neighbors = generate_neighbors(current_route)
        better_neighbor_found = False

        for neighbor in neighbors:
            dist = total_distance(neighbor)
            if dist < current_distance:
                current_route = neighbor
                current_distance = dist
                better_neighbor_found = True
                break 

        if not better_neighbor_found:
            break 

    return current_route, current_distance

locations = [(0, 0), (2, 3), (5, 4), (1, 1), (7, 2)]

best_route, total_dist = hill_climb(locations)
print("Optimized Route:", best_route)
print("Total Distance:", total_dist)


Optimized Route: [(5, 4), (2, 3), (1, 1), (0, 0), (7, 2)]
Total Distance: 16.921096214067973


## Q3

In [5]:
import random
import math

def distance(p1, p2):
    return math.hypot(p1[0] - p2[0], p1[1] - p2[1])

def total_distance(route):
    return sum(distance(route[i], route[(i + 1) % len(route)]) for i in range(len(route)))

def create_route(cities):
    route = cities[:]
    random.shuffle(route)
    return route

def crossover(parent1, parent2):
    size = len(parent1)
    a, b = sorted(random.sample(range(size), 2))
    child = [None] * size
    child[a:b] = parent1[a:b]
    
    pointer = 0
    for city in parent2:
        if city not in child:
            while child[pointer] is not None:
                pointer += 1
            child[pointer] = city
    return child

def mutate(route, mutation_rate=0.01):
    for i in range(len(route)):
        if random.random() < mutation_rate:
            j = random.randint(0, len(route) - 1)
            route[i], route[j] = route[j], route[i]

def tournament_selection(population, scores, k=3):
    selected = random.sample(list(zip(population, scores)), k)
    selected.sort(key=lambda x: x[1])
    return selected[0][0]

def genetic_algorithm(cities, population_size=100, generations=500, mutation_rate=0.01):
    population = [create_route(cities) for _ in range(population_size)]
    best_route = None
    best_score = float('inf')

    for generation in range(generations):
        scores = [total_distance(route) for route in population]
        gen_best_idx = scores.index(min(scores))
        if scores[gen_best_idx] < best_score:
            best_score = scores[gen_best_idx]
            best_route = population[gen_best_idx]

        new_population = [best_route]  # elitism
        while len(new_population) < population_size:
            parent1 = tournament_selection(population, scores)
            parent2 = tournament_selection(population, scores)
            child = crossover(parent1, parent2)
            mutate(child, mutation_rate)
            new_population.append(child)

        population = new_population

    return best_route, best_score

if __name__ == "__main__":
    cities = [(random.randint(0, 100), random.randint(0, 100)) for _ in range(10)]
    
    best_route, best_distance = genetic_algorithm(cities)

    print("Best Route:")
    for city in best_route:
        print(city)
    print("\nTotal Distance:", best_distance)


Best Route:
(67, 1)
(93, 41)
(91, 89)
(83, 88)
(64, 89)
(22, 100)
(17, 77)
(9, 42)
(7, 16)
(32, 27)

Total Distance: 322.68435203168866
